In [5]:
import pygame
import random
from pygame.sprite import Group
import math

pygame.init()

# Classe pour representer le jeu
class Game():

    def __init__(self):
        # definir si le jeu a commence
        self.is_playing = False
        # generer notre joueur
        self.all_players = pygame.sprite.Group()
        self.player = Player(self)
        self.all_players.add(self.player)
        # generer l'evenement
        self.comet_even = CometFallEvent()
        # groupe de monstres
        self.all_monsters = pygame.sprite.Group()
        self.pressed = {}

    def start(self):
        self.is_playing = True
        self.spawn_monster()
        self.spawn_monster()
    
    def game_over(self):
        # remettre le jeu a neuf, retirer les monstres, remettre le joueur a 100 de vie, jeu en attente
        self.all_monsters = pygame.sprite.Group()
        self.player.health = self.player.max_health
        self.is_playing = False
    
    def update(self, screen):
        # appliquer l'image du joueur
        screen.blit(self.player.image,self.player.rect)

        # actualiser la barre de vie du joueur
        self.player.update_health_bar(screen)

        # actualiser la barre d'evenement du jeu
        self.comet_even.update_bar(screen)

        # recuperer les projectiles d'un joueur
        for projectile in self.player.all_projectiles:
            projectile.move()

        # recuperer les monstres
        for monster in game.all_monsters:
            monster.forward()
            monster.update_health_bar(screen)

        # recuperer les cometes
        for comet in self.comet_even.all_comets:
            comet.fall()

        # appliquer l'image du projectile du joueur
        self.player.all_projectiles.draw(screen)

        # appliquer l'image de l'ensemble des monstres
        self.all_monsters.draw(screen) 

        # appliquer l'image de l'ensemble des cometes
        self.comet_even.all_comets.draw(screen)

        if self.pressed.get(pygame.K_RIGHT) and self.player.rect.x + self.player.rect.width<= screen.get_width():
            self.player.move_right()
        if self.pressed.get(pygame.K_LEFT) and self.player.rect.x >= -30:
            self.player.move_left()
    
    def check_collision(self, sprite, group):
        return pygame.sprite.spritecollide(sprite, group, False, pygame.sprite.collide_mask)

    def spawn_monster(self):
        monster = Monster(self)
        self.all_monsters.add(monster)

# Classe pour representer le joueur
class Player(pygame.sprite.Sprite):

    def __init__(self, game):
        super().__init__()
        self.game = game
        self.health = 100
        self.max_health = 100
        self.velocity = 2
        self.attack = 10
        self.all_projectiles = pygame.sprite.Group()
        self.image = pygame.image.load("player.png")
        self.rect = self.image.get_rect()
        self.rect.x = 300
        self.rect.y = 450

    def damage(self,amount):
        # Infliger les degats
        if self.health - amount > amount:
            self.health -= amount
        else:
            # si le joueur n'a plus de points de vie
            self.game.game_over()
    
    def update_health_bar(self, surface):
        # dessiner la barre de vie
        pygame.draw.rect(surface, (60,63,60), [self.rect.x + 50,self.rect.y + 20,self.max_health,5])
        pygame.draw.rect(surface, (111,210,46), [self.rect.x + 50,self.rect.y + 20,self.health,5])
    
    def launch_projectile(self):
        # Creer une nouvelle instance de la classe projectile
            self.all_projectiles.add(Projectile(self))
        
    def move_right(self):
        # si le joueur n'entre pas en collision avec un monstre
        if not self.game.check_collision(self, self.game.all_monsters):
            self.rect.x += self.velocity

    def move_left(self):
        self.rect.x -= self.velocity

# Classe pour representer le projectile du joueur
class Projectile(pygame.sprite.Sprite):

    def __init__(self,player):
        super().__init__()
        self.velocity = 3
        self.player = player
        self.image = pygame.image.load("projectile.png")
        self.image = pygame.transform.scale(self.image,(50,50))
        self.rect = self.image.get_rect()
        self.rect.x = player.rect.x + 130
        self.rect.y = player.rect.y + 80
        self.origin_image = self.image
        self.angle = 0

    def rotate(self):
        self.angle += 12
        self.image = pygame.transform.rotozoom(self.origin_image, self.angle, 1)
        self.rect = self.image.get_rect(center=self.rect.center)
    
    def remove(self):
        self.player.all_projectiles.remove(self)
    
    def move(self):

        self.rect.x += self.velocity
        self.rotate()

        # verifier si le projectile entre en collision avec un monstre
        for monster in self.player.game.check_collision(self, self.player.game.all_monsters):
            # supprimer le projectile
            self.remove()
            # ingliger des degats
            monster.damage(self.player.attack)

        # verifier si le projectile est sorti de l'ecran
        if self.rect.x > 1080:
            # supprimer le projectile
            self.remove()

# Classe pour representer les monstres
class Monster(pygame.sprite.Sprite): 
    
    def __init__(self, game):
        super().__init__()
        self.game = game
        self.health = 100
        self.max_health = 100
        self.attack = 2
        self.image = pygame.image.load("mummy.png")
        self.rect = self.image.get_rect()
        self.rect.x = 1000 + random.randint(0,300)
        self.rect.y = 490 
        self.velocity = random.randint(1,3)

    def damage(self,amount):
        # Infliger les degats
        self.health -= amount

        # verifier si son nouveau npmbre de points de vie est inferieur a 0
        if self.health <= 0:
            # reapparaitre comme un nouveau monstre
            self.rect.x = 1000 + random.randint(0,300)
            self.health = self.max_health

    def update_health_bar(self, surface):
        # definir une couleur pour notre jauge de vie (vert clair)
        bar_color = (111,210,46)
        # definir une couleur pour l'arriere plan de la jauge (gris fonce)
        back_bar_color = (60,63,60)

        # definir la position, la largeur et l'epaisseur de la jauge
        bar_position = [self.rect.x + 10,self.rect.y - 20,self.health,5]
        # definir la position de l'arriere plan de la jauge
        back_bar_position = [self.rect.x + 10,self.rect.y - 20,self.max_health,5]

        # dessiner la barre de vie
        pygame.draw.rect(surface, back_bar_color, back_bar_position)
        pygame.draw.rect(surface, bar_color, bar_position)
        
    
    def forward(self):
        # si le monstre n'entre pas en collision avec le joueur
        if not self.game.check_collision(self, self.game.all_players):
            self.rect.x -= self.velocity

        # si le monstre entre en collision avec le joueur
        else:
            # infliger des degats au joueur
            self.game.player.damage(self.attack)

# Classe pour gerer la pluie de cometes
class CometFallEvent(pygame.sprite.Sprite):

    # lors du chargement -> creer un compteur
    def __init__(self):
        self.percent = 0
        self.percent_speed = 5

        # definir un groupe de sprite pour stocker nos cometes
        self.all_comets = pygame.sprite.Group()

    def add_percent(self):
        self.percent += self.percent_speed / 100
    
    def reset_percent(self):
        self.percent = 0

    def meteor_fall(self):
        # faire apparaitre une comete
        self.all_comets.add(Comet())
    
    def attempt_fall(self):
        # la jauge d'evenement est totalement chargee
        if self.percent >= 100:
            self.meteor_fall()
            print("Pluie de cometes!")
            self.reset_percent()

    def update_bar(self,surface):
        # ajouter du pourcentage a la barre
        self.add_percent()

        # appel de la methode pour essayer de declencher la pluie de cometes
        self.attempt_fall()
        
        # barre noire en arriere plan
        pygame.draw.rect(surface, (0,0,0),[
            0,                      # axe des x
            surface.get_height()-50,   # axe des y
            surface.get_width(),      # longueur de la fenetre
            10                      # epaisseur
            ])
        # barre rouge de la jauge d'evenement
        pygame.draw.rect(surface, (230,230,0),[
            0,                      # axe des x
            surface.get_height()-50,   # axe des y
            (surface.get_width()/100)*self.percent,      # longueur de la fenetre
            10                      # epaisseur
            ])
        
# Classe pour gerer les cometes
class Comet(pygame.sprite.Sprite):

    def __init__(self):
        super().__init__()
        # definir l'image associee a cette comete
        self.image = pygame.image.load("comet.png")
        self.rect = self.image.get_rect()
        self.velocity = 1

    def fall(self):
        self.rect.y += self.velocity
        
# main 

pygame.display.set_caption("Comet Fall Game")
screen = pygame.display.set_mode((1080,720))

background = pygame.image.load("bg.jpg")

# importer la banniere
banner = pygame.image.load("banner.png")
banner = pygame.transform.scale(banner,(500,500))
banner_rect = banner.get_rect()
banner_rect.x = math.ceil(screen.get_width()/4)
banner_rect.y = math.ceil(screen.get_height()/4)

# importer le bouton pour lancer la partie
play_button = pygame.image.load("button.png")
play_button = pygame.transform.scale(play_button,(400,150))
play_button_rect = play_button.get_rect()
play_button_rect.x = math.ceil(screen.get_width()/3.33)
play_button_rect.y = math.ceil(screen.get_width()/3.33) -250

# charger le jeu
game = Game()

running = True

while running:

    # appliquer la fenetre de jeu
    screen.blit(background,(0,-200))

    # verifier si le jeu a commence
    if game.is_playing:
        # declencher les instructions de la partie
        game.update(screen)
    # verifier si le jeu n'a pas commence
    else:
        # ajouter l'ecran de bienvenue
        screen.blit(banner,(banner_rect.x,banner_rect.y))
        # ajouter le bouton
        screen.blit(play_button,(play_button_rect.x,play_button_rect.y))

    print(game.player.rect.x)

    pygame.display.flip()

    for event in pygame.event.get():    # liste des evenements
        if event.type == pygame.QUIT:   # constante de la fermeture de la fenetre
            running = False
            pygame.quit()

        # detecter si une touche a ete relachee
        elif event.type == pygame.KEYDOWN:
            # quelle touche a ete utilisee
            game.pressed[event.key] = True

            # detecter si la touche espace a ete utilisee pour lancer un projectile
            if event.key == pygame.K_SPACE:
                game.player.launch_projectile()

        elif event.type == pygame.KEYUP:
            # quelle touche a ete relachee
            game.pressed[event.key] = False

        elif event.type == pygame.MOUSEBUTTONDOWN:
            # verifier si la souris est en collision avec le bouton PLAY
            if play_button_rect.collidepoint(event.pos):
                game.start()
        


300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
